# Classification with Naive Bayes and SVM

## Naive Bayes

### Preprocessing

In [ ]:
import pandas as pd
import nltk
import os
import re
import string
import numpy as np

from sklearn import datasets, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
df_train = pd.DataFrame(columns=['words', 'sentiment'])
df_test = pd.DataFrame(columns=['words', 'sentiment'])

In [ ]:
sw_dir = './Data/6/sw.txt'
stop_words = []
with open(sw_dir) as f:
    text = f.readlines()
    for word in text:
        stop_words.append(re.findall('\S+', word)[0])

# adding br and empty string to stop words
stop_words.append('br')
stop_words.append('')

In [ ]:
# create dataset from text files
train_pos_dir = './Data/6/aclImdb/train/pos'
for filename in os.listdir(train_pos_dir):
    with open(os.path.join(train_pos_dir, filename)) as f:
        text = f.readlines()[0]
        df_train = df_train.append({'words': text, 'sentiment': 1},
                                   ignore_index=True)

train_neg_dir = './Data/6/aclImdb/train/neg'
for filename in os.listdir(train_neg_dir):
    with open(os.path.join(train_neg_dir, filename)) as f:
        text = f.readlines()[0]
        df_train = df_train.append({'words': text, 'sentiment': 0},
                                   ignore_index=True)

test_pos_dir = './Data/6/aclImdb/test/pos'
for filename in os.listdir(test_pos_dir):
    with open(os.path.join(test_pos_dir, filename)) as f:
        text = f.readlines()[0]
        df_test = df_test.append({'words': text, 'sentiment': 1},
                                   ignore_index=True)

test_neg_dir = './Data/6/aclImdb/test/neg'
for filename in os.listdir(test_neg_dir):
    with open(os.path.join(test_neg_dir, filename)) as f:
        text = f.readlines()[0]
        df_test = df_test.append({'words': text, 'sentiment': 0},
                                   ignore_index=True)

In [ ]:
def remove_punct(text):
    
    def change(ch):
        if ch in string.punctuation or ch.isdigit():
            return " "
        else:
            return ch
    
    no_punct = "".join([change(ch) for ch in text])
    return no_punct

# df_train['words'] = df_train['words'].apply(lambda x: remove_punct(x))


In [ ]:
def tokenize(text):
    tokens = re.split('\W+', text)
    return tokens

# df_train['words'] = df_train['words'].apply(lambda x: tokenize(x.lower()))


In [ ]:
def remove_sw(tokens):
    text = [w for w in tokens if w not in stop_words]
    return text

# df_train['words'] = df_train['words'].apply(lambda x: remove_sw(x))

In [ ]:
def remove_short(tokens):
    text = [w for w in tokens if len(w)>2]
    return text

# df_train['words'] = df_train['words'].apply(lambda x: remove_short(x))

There are several ways to get root of tokens like stemming and lemmatizing. stemming is faster and lemmatizing is more precise.

In [ ]:
wn = nltk.WordNetLemmatizer()
def lemmatizing(tokens):
    text = [wn.lemmatize(w) for w in tokens]
    return text

ps = nltk.stem.PorterStemmer()
def stemming(tokens):
    text = [ps.stem(w) for w in tokens]
    return text

# df_train['words'] = df_train['words'].apply(lambda x: lemmatizing(x))
# df_train['words'] = df_train['words'].apply(lambda x: stemming(x))

In [ ]:
def clean_text(text):
    text = remove_punct(text)
    text = tokenize(text)
    text = remove_sw(text)
    text = remove_short(text)
    # text = lemmatizing(text)
    text = stemming(text)
    return text

count_vect = CountVectorizer(analyzer=clean_text,
                             lowercase=True,
                             binary=True)
X_train = count_vect.fit_transform(df_train['words'])
y_train = df_train['sentiment'].to_numpy(dtype='int')

X_test = count_vect.transform(df_test['words'])
y_test = df_train['sentiment'].to_numpy(dtype='int')
# print(count_vect.get_feature_names())

### Classification

In [ ]:
clf = MultinomialNB(alpha=100)
clf = clf.fit(X_train, y_train)

In [ ]:
# clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

acc_tr = clf.score(X_train, y_train)
acc_te = clf.score(X_test, y_test)

print("train accuracy: {}%".format(acc_tr*100))
print("test accuracy: {}%".format(acc_te*100))

In [ ]:
sns.heatmap(cm, annot=True, fmt='d')

### Laplace smoothing

In [ ]:
alphas = [10**x for x in range(-4, 4)]

accs_tr = []
accs_te = []

for alpha in alphas:
    cls = MultinomialNB(alpha=alpha)
    cls = cls.fit(X_train, y_train)
    accs_tr.append(cls.score(X_train, y_train))
    accs_te.append(cls.score(X_test, y_test))
    
plt.plot(alphas, accs_tr)
plt.plot(alphas, accs_te)
plt.xlabel('alpha')
plt.ylabel('accuracy')
plt.legend(['train accuracy', 'test accuracy'])
plt.xscale('log')

## SVM Classifier

In [ ]:
def make_meshgrid(x, y, h=.02):
    
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

In [ ]:
def plot_contours(ax, clf, xx, yy, **params):
    
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

In [ ]:
iris = datasets.load_iris()

X = iris.data[:, :2]
y = iris.target

In [ ]:
models = (svm.SVC(kernel='linear', C=1.0),
          svm.LinearSVC(C=1.0, max_iter=10000),
          svm.SVC(kernel='rbf', gamma=0.7, C=1.0),
          svm.SVC(kernel='poly', degree=3, gamma='auto', C=1.0))
models = (clf.fit(X, y) for clf in models)

In [ ]:
titles = ('SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel')

fig, sub = plt.subplots(2, 2, figsize=(12, 8))
plt.subplots_adjust(wspace=0.4, hspace=0.4)

X0, X1 = X[:, 0], X[:, 1]
xx, yy = make_meshgrid(X0, X1)

for clf, title, ax in zip(models, titles, sub.flatten()):
    plot_contours(ax, clf, xx, yy,
                  cmap=plt.cm.coolwarm, alpha=0.8)
    ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('Sepal length')
    ax.set_ylabel('Sepal width')
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_title(title)

plt.show()